In [72]:
import pandas as pd
import os

# SAFECAST_DATA_PATH = os.path.join('results', 'first_million.csv')
SAFECAST_DATA_PATH = r"F:\safecast\chunks\filtered.csv"
df = pd.read_csv(SAFECAST_DATA_PATH, nrows=10000000, names=["capture_date", "latitude", "longitude", "value"])

In [73]:
df.head()

,capture_date,latitude,longitude,value
0,2018-10-21 01:00:26.000000,36.041080,140.226816,23.0
1,2018-10-21 01:00:22.000000,37.796306,140.514413,19.0
2,2018-10-21 01:00:16.000000,37.723330,140.476797,15.0
3,2018-10-21 00:59:16.000000,52.444900,13.315000,16.0
4,2018-10-21 01:00:16.000000,37.787500,140.552400,18.0


In [75]:
from gmalthgtparser import HgtParser
from functools import lru_cache

HGTDIR = "hgt_files"


def get_file_name(lat, lon):
    
    @lru_cache(maxsize=1024)
    def _get_file_name(latitude, longitude, ns, ew):
        hgt_file = "%(ns)s%(lat)02d%(ew)s%(lon)03d.hgt" % {'lat': latitude, 'lon': longitude, 'ns': ns, 'ew': ew}
        hgt_file_path = os.path.join(HGTDIR, hgt_file)
        if os.path.isfile(hgt_file_path):
            return hgt_file_path
        else:
            return None
        
    if lat >= 0.0:
        ns = 'N'
    else:
        ns = 'S'
        lat -= 1

    if lon >= 0.0:
        ew = 'E'
    else:
        ew = 'W'
        lon -= 1
    latitude = abs(lat)
    longitude = abs(lon)
    return _get_file_name(latitude, longitude, ns, ew)

In [76]:
def read_elevation_from_file(hgt_file, lat, lon):
    with HgtParser(hgt_file) as parser:
        return parser.get_elevation((lat, lon))[2]

In [77]:
import numpy as np

SAMPLES = 1201

def read_elevation_from_file2(hgt_file, lat, lon):
    
    with open(hgt_file, 'rb') as hgt_data:
        # Each data is 16bit signed integer(i2) - big endian(>)
        elevations = np.fromfile(hgt_data, np.dtype('>i2'), SAMPLES*SAMPLES)\
                                .reshape((SAMPLES, SAMPLES))

        lat_row = int(round((lat - int(lat)) * (SAMPLES - 1), 0))
        if lat_row < 0:
            lat_row = SAMPLES  - 1 + lat_row
        lon_row = int(round((lon - int(lon)) * (SAMPLES - 1), 0))
        if lon_row < 0:
            lon_row = SAMPLES - 1 + lon_row
        return elevations[SAMPLES - 1 - lat_row, lon_row].astype(int)

In [78]:
d = {}

    
def get_elevation_for_coords(lat, lon):
    hgt_file = get_file_name(lat, lon)
    if not hgt_file:
        return -32768
    
    lat_row = int(round((lat - int(lat)) * (SAMPLES - 1), 0))
    if lat_row < 0:
        lat_row = SAMPLES - 1 + lat_row
    lon_row = int(round((lon - int(lon)) * (SAMPLES - 1), 0))
    if lon_row < 0:
        lon_row = SAMPLES - 1 + lon_row
    if hgt_file not in d:
        with open(hgt_file, 'rb') as hgt_data:
            d[hgt_file] = np.fromfile(hgt_data, np.dtype('>i2'), SAMPLES*SAMPLES).reshape((SAMPLES, SAMPLES))
            
    try:
        return d[hgt_file][SAMPLES - 1 - lat_row, lon_row].astype(int)
    except Exception as e:
        print(lat, lon, repr(e))
        return -32768


In [20]:
lat, lon = 52.444900, 13.315000
hgt_file = get_file_name(lat, lon)

In [21]:
%%timeit
read_elevation_from_file(hgt_file, lat, lon)

2.76 ms ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
read_elevation_from_file2(hgt_file, lat, lon)

2.75 ms ± 33.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [79]:
def get_elevation(row):
    lat, lon = row.latitude, row.longitude
    return get_elevation_for_coords(lat, lon)

In [64]:
lat = -33.945945
lon = 18.3923316666667
f = get_file_name(lat, lon)
print(read_elevation_from_file(f, lat, lon))
print(read_elevation_from_file2(f, lat, lon))
lat_row = abs(int(round((lat - int(lat)) * (SAMPLES - 1), 0)))
lon_row = int(round((lon - int(lon)) * (SAMPLES - 1), 0))
lat_row, lon_row

207
207


(1135, 471)

In [63]:
lat = 21.019000
lon = -101.257400
f = get_file_name(lat, lon)
print(read_elevation_from_file(f, lat, lon))
print(read_elevation_from_file2(f, lat, lon))
lat_row = abs(int(round((lat - int(lat)) * (SAMPLES - 1), 0)))
lon_row = int(round((lon - int(lon)) * (SAMPLES - 1), 0))
lat_row, lon_row

2023
2023


(23, -309)

In [80]:
df["elevation"] = df.apply(get_elevation, axis=1)

In [81]:
len(df[df.elevation == -32768])

267481

In [83]:
len(d)

841

In [84]:
filtered_df = df[df.elevation != -32768]

In [85]:
len(filtered_df)

9732519

In [86]:
filtered_df.head()

,capture_date,latitude,longitude,value,elevation
0,2018-10-21 01:00:26.000000,36.041080,140.226816,23.0,31
1,2018-10-21 01:00:22.000000,37.796306,140.514413,19.0,72
2,2018-10-21 01:00:16.000000,37.723330,140.476797,15.0,141
3,2018-10-21 00:59:16.000000,52.444900,13.315000,16.0,47
4,2018-10-21 01:00:16.000000,37.787500,140.552400,18.0,107


In [87]:
filtered_df.to_csv(os.path.join("results", "10_million_with_elevation.csv"), index=False)

In [18]:
path = os.path.join("results", "fukushima_filtered_elevation.csv")
df.to_csv(path, index=False)

In [1]:
import zipfile
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

import requests
from bs4 import BeautifulSoup

files_dir = "hgt_files"


def download_file(url):
    resp = requests.get(url)
    with zipfile.ZipFile(BytesIO(resp.content)) as zip_ref:
        zip_ref.extractall(files_dir)
    return url
        

def download_files():
    file_urls = []
    urls = [
        "https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Africa/",
        "https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Australia/",
        "https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Eurasia/",
        "https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Islands/",
        "https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/North_America/",
        "https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/South_America/"
    ]
    for url in urls:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html.parser")
        links = [url + link["href"] for link in soup.find_all("a") if "hgt.zip" in link["href"]]
        print(len(links), "files in", url)
        file_urls.extend(links)
    
    print(len(file_urls), "files to download")
    with ThreadPoolExecutor() as pool:
        for downloaded_url in pool.map(download_file, file_urls):
            print(downloaded_url)